In [ ]:
pip install "pymilvus[model]"


In [2]:
from pymilvus import MilvusClient

# client = MilvusClient("milvus_demo.db") # sqlite
client = MilvusClient("http://localhost:19530")


In [3]:
if client.has_collection(collection_name="demo_collection"):
    client.drop_collection(collection_name="demo_collection")
client.create_collection(
    collection_name="demo_collection",
    dimension=768,  # The vectors we will use in this demo has 768 dimensions
)


In [4]:
from pymilvus import model


embedding_fn = model.DefaultEmbeddingFunction()

docs = [
    "이성욱은 AI 담당자이다.",
    "Artificial intelligence was founded as an academic discipline in 1956.",
    "Alan Turing was the first person to conduct substantial research in AI.",
    "Born in Maida Vale, London, Turing was raised in southern England.",
]

vectors = embedding_fn.encode_documents(docs)
print("Dim:", embedding_fn.dim, vectors[0].shape)  # Dim: 768 (768,)

data = [
    {"id": i, "vector": vectors[i], "text": docs[i], "subject": "history"}
    for i in range(len(vectors))
]

print("Data has", len(data), "entities, each with fields: ", data[0].keys())
print("Vector dim:", len(data[0]["vector"]))


/Users/passion1014/anaconda3/envs/ragserver/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Dim: 768 (768,)
Data has 4 entities, each with fields:  dict_keys(['id', 'vector', 'text', 'subject'])
Vector dim: 768


In [5]:
res = client.insert(collection_name="demo_collection", data=data)

print(res)


{'insert_count': 4, 'ids': [0, 1, 2, 3]}


In [8]:
query_vectors = embedding_fn.encode_queries(["Who is Alan Turing?"])

res = client.search(
    collection_name="demo_collection",  # target collection
    data=query_vectors,  # query vectors
    limit=2,  # number of returned entities
    output_fields=["text", "subject"],  # specifies fields to be returned
)

print(res)


data: ["[{'id': 3, 'distance': 0.5859944820404053, 'entity': {'text': 'Born in Maida Vale, London, Turing was raised in southern England.', 'subject': 'history'}}, {'id': 2, 'distance': 0.5118255615234375, 'entity': {'text': 'Alan Turing was the first person to conduct substantial research in AI.', 'subject': 'history'}}]"]


In [11]:
from pymilvus import connections, Collection

# Milvus 서버 연결
connections.connect(
    alias="default",
    host="localhost",  # Milvus 서버 주소
    port="19530"       # Milvus 기본 포트 (클러스터 환경에서는 변경 가능)
)

# Collection 객체 가져오기
collection = Collection("demo_collection")

# Flush 실행 (Growing Segment → Sealed Segment로 변환)
collection.flush()

print("Flush completed successfully!")


Flush completed successfully!
